In [17]:
import pandas as pd
import numpy as np
import json
from datetime import datetime, tzinfo, timedelta
from IPython.display import display, HTML

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.offline import plot
from plotly.graph_objs import Scatter, Data, Box, Bar
from plotly.graph_objs import Scattergl
init_notebook_mode() # run at the start of every ipython notebook to use plotly.offline
                     # this injects the plotly.js source files into the notebook
from plotly.tools import FigureFactory as FF
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

%matplotlib inline
import matplotlib as plt
plt.style.use('ggplot')
import cufflinks as cf
cf.set_config_file(offline=True, world_readable=True, theme='ggplot')

In [2]:
# make tables prettier
from IPython.core.display import HTML
css = open("/Users/coreypetty/PycharmProjects/TBP_metrics/styles/style-table.css").read()
HTML('<style>{}</style>'.format(css))

# Get JSON data from EtherScan.io API

Note that EtherScan's API only downloads 10k transactions per call, so we download all pages, and then concatenate them together.

In [470]:
import urllib3, json, certifi
import pandas as pd
import shutil
from api_key import key

def transactions(page=1, filename='sample.json'):
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',  # Force certificate check.
                               ca_certs=certifi.where(),  # Path to the Certifi bundle.
                               )

    dao_address = '0xbb9bc244d798123fde783fcc1c72d3bb8c189413'
    etherscan_url = 'https://api.etherscan.io/api?module=account&action=txlist&address=' + dao_address + '&page=' + str(page) + '&offset=10000&sort=asc&apikey=' + key

    with open(filename, 'w') as f, http.request('GET', etherscan_url, preload_content=False) as r:
        string_r = r.data.decode('utf-8')
        obj = json.loads(string_r)
        json.dump(obj['result'], f, sort_keys=True, indent=4)
    print(obj['message'])

transactions(filename='DAO_page1.json', page=1)
transactions(filename='DAO_page2.json', page=2)
transactions(filename='DAO_page3.json', page=3)
transactions(filename='DAO_page4.json', page=4)
transactions(filename='DAO_page5.json', page=5)
transactions(filename='DAO_page6.json', page=6)
transactions(filename='DAO_page7.json', page=7)
transactions(filename='DAO_page8.json', page=8)

OK
OK
OK
OK
OK
OK
No transactions found
No transactions found


Read in JSON data, concatenate into one DataFrame

In [3]:
trans1 = pd.read_json("DAO_page1.json")
trans2 = pd.read_json("DAO_page2.json")
trans3 = pd.read_json("DAO_page3.json")
trans4 = pd.read_json("DAO_page4.json")
trans5 = pd.read_json("DAO_page5.json")
trans6 = pd.read_json("DAO_page6.json")
trans = pd.concat([trans1, trans2, trans3, trans4, trans5, trans6])

# Clean up the DataFrame

In [4]:
dao_end_time = datetime(2016, 5, 28, 9, 0, 0, 0)
dao_phase_one_end = dao_end_time - timedelta(days=15)
dao_phase_two_end = dao_end_time - timedelta(days=10)

In [5]:
trans['value'] = trans.value.astype(float)
trans['blockNumber'] = trans.blockNumber.astype(int)
trans['value_ether'] = trans.value / 1e18
trans['gasPrice'] = trans.gasPrice.astype(float)
trans['gasUsed'] = trans.gasUsed.astype(float)
trans['value_cum'] = trans.value_ether.cumsum()
trans.cumulativeGasUsed = trans.cumulativeGasUsed.astype(float)
trans = trans.sort_values(by='blockNumber').reset_index(drop=True)

trans.head()

,blockHash,blockNumber,confirmations,contractAddress,cumulativeGasUsed,from,gas,gasPrice,gasUsed,hash,input,nonce,timeStamp,to,transactionIndex,value,value_ether,value_cum
0,0x9a9d1fc252e816fd047b85b6c1028317ecc0bf3db18d...,1429008,138309,,47080.0,0x931f12be72a1d106408af5ced7e1b7af0bc864ae,49900,2.000000e+10,47080.0,0x18aa650060a1434310a05824e1df6e29a6904f29839e...,0x095ea7b3000000000000000000000000fbb1b73c4f0b...,0,2016-04-30 02:44:37,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,0,0.0,0.0,0.0
1,0x9a9d1fc252e816fd047b85b6c1028317ecc0bf3db18d...,1429008,138309,,136160.0,0xb504e60998c6f354a0794abd91d85e8bc8436211,49900,2.000000e+10,47080.0,0xe71a87b83e054f616e8b1f4884b905d9a347b64433bb...,0x095ea7b3000000000000000000000000fbb1b73c4f0b...,0,2016-04-30 02:44:37,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,3,0.0,0.0,0.0
2,0x41a0f3f494329c60c88ae92bf6fc6f35c389e27f53f3...,1429009,138308,,68080.0,0xe0b8bdfddba2b1636c9a975f9e9759169fddee45,49900,2.000000e+10,47080.0,0xc750552b741c499b073dc58d7148c0e8f24c9b4e0a94...,0x095ea7b3000000000000000000000000fbb1b73c4f0b...,0,2016-04-30 02:45:18,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,1,0.0,0.0,0.0
3,0x41a0f3f494329c60c88ae92bf6fc6f35c389e27f53f3...,1429009,138308,,199599.0,0x2c30cdad7abdeedb44657a7b46827fc38af12203,49900,2.000000e+10,47080.0,0x7a47fdb185d89ab843e71d53702b6ea172378007cf12...,0x095ea7b3000000000000000000000000fbb1b73c4f0b...,0,2016-04-30 02:45:18,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,5,0.0,0.0,0.0
4,0x41a0f3f494329c60c88ae92bf6fc6f35c389e27f53f3...,1429009,138308,,246679.0,0xd97fdc4af28008c219639401f661369b704e7bbc,49900,2.000000e+10,47080.0,0x5ab9ac02611a1f0652706e4f43515840d1dc33a22b87...,0x095ea7b3000000000000000000000000fbb1b73c4f0b...,0,2016-04-30 02:45:18,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,6,0.0,0.0,0.0


# Breaking up into DAO phases

In [6]:
dao_end_time = datetime(2016, 5, 28, 9, 0, 0, 0)
dao_phase_one_end = dao_end_time - timedelta(days=14)
dao_phase_two_end = dao_end_time - timedelta(days=5)

In [7]:
print(dao_phase_one_end.strftime("%d %b %Y %I:%M:%S %p %Z") )
print(dao_phase_two_end.strftime("%d %b %Y %I:%M:%S %p %Z") )
print(dao_end_time.strftime("%d %b %Y %I:%M:%S %p %Z") )

14 May 2016 09:00:00 AM 
23 May 2016 09:00:00 AM 
28 May 2016 09:00:00 AM 


In [8]:
def token_rate(timeStamp):
    token_rate = [1.0 + 0.05*(x+1) for x in range(10)]
    days = [datetime(2016, 5, 28, 9, 0, 0, 0) - timedelta(days=13) + timedelta(days=1)*x for x in range(10)]
    
    for rate, day in zip(token_rate,days):
        if timeStamp < days[0] - timedelta(days=1):
            return 1.0
        elif day < timeStamp < day + timedelta(days=1):
            return rate
    return 1.5

trans['token_rate'] = trans.timeStamp.apply(token_rate)
trans['tokens'] = trans.value_ether / trans.token_rate * 100

In [9]:
# Note the final block of the 1 ETH to 100 tokens
blockEnd_phaseOne = 1520860
# Separate the transactions into parts, before and after token price change
before = trans[trans.timeStamp <= dao_phase_one_end]
after = trans[trans.timeStamp > dao_phase_one_end]

In [10]:
before.timeStamp.max()

Timestamp('2016-05-14 08:59:47')

# Total Ether

Before Token Price Change

In [11]:
EthToBtc = 0.025
BtcToUsd = 456.25
total_ether_before = before.value_ether.sum()
total_tokens_before = before.tokens.sum()
print('ETH {} '.format(total_ether_before))
print('BTC {}'.format(total_ether_before * EthToBtc))
print('USD {}'.format(total_ether_before * EthToBtc * BtcToUsd))

ETH 9439257.50737325 
BTC 235981.43768433123
USD 107666530.94347613


After Token Price Change

In [12]:
total_ether_after = after.value_ether.sum()
total_tokens_after = after.tokens.sum()
print('ETH {} '.format(total_ether_after))
print('BTC {}'.format(total_ether_after * EthToBtc))
print('USD {}'.format(total_ether_after * EthToBtc * BtcToUsd))

ETH 1823583.481293998 
BTC 45589.587032349955
USD 20800249.08350967


Total Ether (Together)

In [13]:
total_ether = total_ether_before + total_ether_after
total_tokens = total_tokens_before + total_tokens_after
print('ETH {} '.format(total_ether))
print('BTC {}'.format(total_ether * EthToBtc))
print('USD {}'.format(total_ether * EthToBtc * BtcToUsd))

ETH 11262840.988667248 
BTC 281571.0247166812
USD 128466780.02698581


# Total Transactions and Unique Addresses

Total Transactions

In [14]:
total_transactions_before = len(before)
print('Before Token Price Change: {}'.format(total_transactions_before))
total_transactions_after = len(after)
print('After Token Price Change:  {}'.format(total_transactions_after))
total_transactions = len(trans)
print('Total:                     {}'.format(total_transactions))

Before Token Price Change: 37182
After Token Price Change:  19166
Total:                     56348


Unique Addresses

In [15]:
unique_address_before = len(before['from'].unique())
unique_address_after = len(after['from'].unique())
unique_address = len(trans['from'].unique())
print('before: {}'.format(unique_address_before))
print('after:  {}   Note: not cross referenced with "before"'.format(unique_address_after))
print('total:  {}'.format(unique_address))

before: 12439
after:  7287   Note: not cross referenced with "before"
total:  18385


# Cumulative Transaction Volume

In [18]:
trace = go.Scatter(
    x=trans.timeStamp, 
    y=trans.value_cum,
    name='Transaction Values',
)
data = [trace]
layout = go.Layout(
    title="Cummulative Transaction Volume",
    xaxis=go.XAxis(
        title='Block number',
    ),
    yaxis=go.YAxis(
        title='Amount inversted (Ether)',
    ),
)
fig = go.Figure(data=data, layout=layout)

iplot(fig)
# py.iplot(fig, filename='DAO_plots/cummulative_trans')

# Accumulate (Ether, Number Transactions) Into Unique Addreses

Before

In [19]:
cum_ether = pd.DataFrame(before.groupby('from').value_ether.sum().reset_index(), columns=['from', 'value_ether'])
cum_trans = before.groupby('from').size().sort_values().reset_index()
cum_trans.columns = ['from', 'num_trans']
grouped_before = pd.merge(cum_ether, cum_trans, on='from')

After

In [20]:
cum_ether = pd.DataFrame(after.groupby('from').value_ether.sum().reset_index(), columns=['from', 'value_ether'])
cum_trans = after.groupby('from').size().sort_values().reset_index()
cum_trans.columns = ['from', 'num_trans']
grouped_after = pd.merge(cum_ether, cum_trans, on='from')

Total

In [21]:
cum_ether = pd.DataFrame(trans.groupby('from').value_ether.sum().reset_index(), columns=['from', 'value_ether'])
cum_tokens = pd.DataFrame(trans.groupby('from').tokens.sum().reset_index(), columns=['from','tokens'])
cum_trans = trans.groupby('from').size().sort_values().reset_index()
cum_trans.columns = ['from', 'num_trans']
grouped = pd.merge(cum_ether, cum_trans, on='from')
grouped = pd.merge(grouped, cum_tokens, on='from')

## Looking at High Rollers

In [485]:
grouped[grouped.value_ether > 75000].sort_values(by='value_ether')

,from,value_ether,num_trans,tokens
13993,0xc2975b244195b1a5b2b7ae982e12b9bc32daa0b3,80000.000000,6,8.000000e+06
10816,0x961914a2f166f2e2e4ecdc691b4a9272588a8aee,80317.972216,6,8.031797e+06
7849,0x6c57430aac511421c811871fe5375874d0d2742a,81500.000000,4,8.083333e+06
9881,0x88bbf6f5ba896262d80dbbe3597d09c817fd1475,82000.000000,6,7.966667e+06
5947,0x512b91bbfaa9e581ef683fc90d9db22a8f49f48b,100000.000000,1,1.000000e+07
18125,0xfc361105dd90f9ede566499d69e9130395f12ac8,100000.000000,1,6.666667e+06
3995,0x36bf43ff35df90908824336c9b31ce33067e2f50,100000.000000,2,1.000000e+07
15919,0xddd31eb39d56d51b50172884bd2b88e1f6264f95,104048.000000,17,9.410226e+06
16174,0xe10c540088113fa6ec00b4b2c8824f8796e96ec4,110000.000000,2,9.333333e+06
18092,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,177352.234326,4199,1.641865e+07


In [486]:
grouped[grouped.num_trans > 50].sort_values(by='num_trans')

,from,value_ether,num_trans,tokens
8242,0x71cef5157b584fe7c90306aba7a3a5260688c78c,2845.000000,61,2.845000e+05
5582,0x4beb565de3a30aa598f8f0bfe9363f6d7542c321,3.830000,98,3.064000e+02
6117,0x536fef0b5ae77486a5795048a659d689cd787c36,4961.275535,123,4.361337e+05
1665,0x167a9333bf582556f35bd4d16a7e80e191aa6476,22943.484749,188,1.860892e+06
4183,0x391fc9c9663a61eeb3dc267e4fe9d4188ee7c1b8,1972.408300,206,1.972408e+05
3021,0x2910543af39aba0cd09dbb2d50200b3e800a63d2,35522.703030,275,3.377323e+06
7672,0x69ea6b31ef305d6b99bb2d4c9d99456fa108b02a,7023.989027,341,5.893874e+05
4707,0x40b9b889a21ff1534d018d71dc406122ebcf3f5a,194191.103618,843,1.941911e+07
11416,0x9e6316f44baeeee5d41a1070516cc5fa47baf227,39780.437648,975,3.585881e+06
17326,0xf0e42abda410cefb5b4dc4de92a3de5b309e02f2,0.000111,1004,1.111111e-02


Known Exchange Addresses

In [22]:
exchanges = dict(
    poloniex = '0xb794f5ea0ba39494ce839613fffba74279579268',
    polo_2 = '0x32be343b94f860124dc4fee278fdcbd38c102d88',
    kraken = '0x2910543af39aba0cd09dbb2d50200b3e800a63d2',
    bitfinex = '0xcafb10ee663f465f9d10588ac44ed20ed608c11e',  
    gatecoin_hacked = '0x40b9b889a21ff1534d018d71dc406122ebcf3f5a',
)

In [23]:
def is_exchange(add_dict:dict, df):
    for exchange, address in add_dict.items():
        if address in df['from'].values:
            print("{}, {}, {}".format(exchange, df[df['from'] == address].value_ether.values, df[df['from']==address].num_trans.values))
            
is_exchange(exchanges, grouped)

kraken, [ 35522.70303], [275]
gatecoin_hacked, [ 194191.103618], [843]
polo_2, [ 697929.88320793], [5826]


In [24]:
grouped[(grouped.value_ether > 35000) & (grouped.num_trans > 200)]

,from,value_ether,num_trans,tokens
3021,0x2910543af39aba0cd09dbb2d50200b3e800a63d2,35522.703030,275,3.377323e+06
3699,0x32be343b94f860124dc4fee278fdcbd38c102d88,697929.883208,5826,6.530247e+07
4707,0x40b9b889a21ff1534d018d71dc406122ebcf3f5a,194191.103618,843,1.941911e+07
11416,0x9e6316f44baeeee5d41a1070516cc5fa47baf227,39780.437648,975,3.585881e+06
18092,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,177352.234326,4199,1.641865e+07


# Looking at the distribution of wealth

In [25]:
def label_exp_group(value):
    labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8']
    for label in labels:
        if value <= 10**float(label):
            return label

trans['exp_group'] = trans.value_ether.apply(label_exp_group)
grouped['exp_group'] = grouped.value_ether.apply(label_exp_group)
grouped_ether = grouped.groupby('exp_group').sum()
grouped_accounts = grouped.groupby('exp_group').size()
grouped_accounts = pd.DataFrame(grouped_accounts, columns=['num_addresses'])
grouped_ether['value_perc'] = grouped_ether.value_ether / total_ether * 100
grouped_ether['token_perc'] = grouped_ether.tokens / total_tokens * 100
grouped_accounts['percentage'] = grouped_accounts.num_addresses / unique_address * 100

In [490]:
grouped

,value_ether,num_trans,tokens,value_perc,token_perc
exp_group,,,,,
0,7.450550e+02,5879,6.661073e+04,0.006615,0.006144
1,1.519832e+04,4894,1.397592e+06,0.134942,0.128905
2,2.497362e+05,12938,2.333426e+07,2.217346,2.152197
3,1.443708e+06,13755,1.381774e+08,12.818328,12.744563
4,3.412748e+06,5735,3.316326e+08,30.300955,30.587589
5,3.909277e+06,2233,3.777280e+08,34.709508,34.839121
6,2.231428e+06,10914,2.118700e+08,19.812305,19.541481


In [27]:
trace = go.Bar( 
    y=grouped_accounts.percentage,
    name='Unique Addresses',
)
trace2 = go.Bar( 
    y=grouped_ether.token_perc,
    name='Total Tokens Held',
)
data = Data([trace, trace2])
layout = go.Layout(
    xaxis=go.XAxis(
#         title='Investor Group',
#         ticktext = [
#                 "$0 < x < 10^0$",
#                 "$10^0 < x < 10^1$",
#                 "$10^1 < x < 10^2$",
#                 "$10^2 < x < 10^3$",
#                 "$10^3 < x < 10^4$",
#                 "$10^4 < x < 10^5$",
#                 "$10^5 < x < 10^6$",
#             ],
        tickvals = [ 0, 1, 2, 3, 4, 5, 6 ]
    ),
    yaxis=go.YAxis(
        title='Percentage',
    ),
    legend=dict(
        x=0,
        y=1
    )
)
fig = go.Figure(data=data, layout=layout)

#iplot(fig)
py.iplot(fig, filename='DAO_plots/PercentageAccountsVsPercentageEther')

In [29]:
df_exc = grouped
df_exc.loc[((df_exc.num_trans > 100) & (df_exc.value_ether > 30000)),'exp_group'] = 'exchange'
exc_ether = df_exc.groupby('exp_group').sum()
exc_accounts = df_exc.groupby('exp_group').size()
exc_accounts = pd.DataFrame(exc_accounts, columns=['num_addresses'])
exc_ether['value_perc'] = exc_ether.value_ether / total_ether * 100
exc_ether['token_perc'] = exc_ether.tokens / total_tokens * 100
exc_accounts['percentage'] = exc_accounts.num_addresses / unique_address * 100

In [30]:
exc_table = df_exc[df_exc.exp_group == 'exchange']
exc_table['token_perc'] = exc_table.tokens / total_tokens * 100
exc_table['ether_perc'] = exc_table.value_ether / total_ether * 100
exc_table['exchange'] = ['kraken', 'poloniex', 'gatecoin', '??', '??']
exc_table = exc_table[['exchange', 'from', 'value_ether', 'ether_perc', 'tokens', 'token_perc']]
exc_table.columns = ['Exchange', 'Address', 'Ether Invested', 'Ether %', 'Tokens', "Token %"]
exc_table = exc_table.reset_index(drop=True)
exc_table['Ether Invested'] = exc_table['Ether Invested'].map('{:,.2f}'.format)
exc_table['Ether %'] = exc_table['Ether %'].map('{:,.2f}'.format)
exc_table['Tokens'] = exc_table['Tokens'].map('{:,.2f}'.format)
exc_table['Token %'] = exc_table['Token %'].map('{:,.2f}'.format)
exc_table

/Users/coreypetty/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/coreypetty/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/coreypetty/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

,Exchange,Address,Ether Invested,Ether %,Tokens,Token %
0,kraken,0x2910543af39aba0cd09dbb2d50200b3e800a63d2,"35,522.70",0.32,"3,377,323.35",0.31
1,poloniex,0x32be343b94f860124dc4fee278fdcbd38c102d88,"697,929.88",6.20,"65,302,466.70",6.02
2,gatecoin,0x40b9b889a21ff1534d018d71dc406122ebcf3f5a,"194,191.10",1.72,"19,419,110.36",1.79
3,??,0x9e6316f44baeeee5d41a1070516cc5fa47baf227,"39,780.44",0.35,"3,585,880.66",0.33
4,??,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,"177,352.23",1.57,"16,418,649.86",1.51


In [493]:
df = exc_ether.join(exc_accounts)

In [31]:
trace = go.Bar( 
    y=exc_accounts.percentage,
    name='Unique Addresses',
)
trace2 = go.Bar( 
    y=exc_ether.token_perc,
    name='Total Tokens Held',
)
data = Data([trace, trace2])
layout = go.Layout(
    xaxis=go.XAxis(
        title='Investor Group',
        ticktext = [
                "$0 < x < 10^0$",
                "$10^0 < x < 10^1$",
                "$10^1 < x < 10^2$",
                "$10^2 < x < 10^3$",
                "$10^3 < x < 10^4$",
                "$10^4 < x < 10^5$",
                "$10^5 < x < 10^6$",
                "Exchange"
            ],
        tickvals = [ 0, 1, 2, 3, 4, 5, 6, 7 ]
    ),
    yaxis=go.YAxis(
        title='Percentage',
    ),
    legend=dict(
        x=0,
        y=1
    ),
#    barmode='stack'
)
fig = go.Figure(data=data, layout=layout)

# iplot(fig)
py.iplot(fig, filename='DAO_plots/PercentageAccountsVsPercentageEther_exchange')

# Looking at number transactions per day

In [76]:
group_block = trans

In [75]:
trace = go.Bar(
    x=group_block.set_index('timeStamp').resample('D').reset_index().timeStamp,
    y=group_block.set_index('timeStamp').resample('D').tokens.sum(),
    name='Tokens Dispersed'
    )
trace2 = go.Bar(
    x=group_block.set_index('timeStamp').resample('D').reset_index().timeStamp,
    y=group_block.set_index('timeStamp').resample('D').size(),
    name='Transactions Made'
    )
data = Data([trace,trace2])
fig = tools.make_subplots(rows=2, cols=1,
                          specs=[[{}], [{}]],
                          )
fig.append_trace(trace, 1, 1)
fig.append_trace(trace2, 2, 1)
fig['layout'].update(legend=dict(x=0,y=1))
fig['layout']['xaxis2'].update(title='Date')
fig['layout']['yaxis1'].update(title='Tokens Dispersed')
fig['layout']['yaxis2'].update(title='Number of Transactions')
#iplot(fig)
py.iplot(fig, filename='DAO_plots/stackedTokensTransactions')

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



/Users/coreypetty/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning:

.resample() is now a deferred operation
use .resample(...).mean() instead of .resample(...)

/Users/coreypetty/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning:

.resample() is now a deferred operation
use .resample(...).mean() instead of .resample(...)

